In [1]:
# Importing libraries and packages

import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import time
import random

In [2]:
# random time function

def sec(): 
    time = random.randrange(1, 10)
    return time

In [3]:
# options.headless - to run the chrome application in the background

options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")
#options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36")

In [4]:
# Installing Webdriver for Chrome

driver = webdriver.Chrome('C:/Users/lxf12/web_scraping/chromedriver', chrome_options=options)
driver.get('https://www.lowes.com/pd/Project-Source-Project-Source-Pro-Flush-2-Piece-Single-Flush-Elongated-Chair-Height-White-1-28-GPF/1003066162')

In [5]:
# Saving 'Lowes' website in url

url = driver.current_url
print("URL:", url)

URL: https://www.lowes.com/pd/Project-Source-Project-Source-Pro-Flush-2-Piece-Single-Flush-Elongated-Chair-Height-White-1-28-GPF/1003066162


In [6]:
# Scraping Product Name using XPATH

product_name = driver.find_element(By.XPATH, './/*[@id="pdp-lpd"]/div/div[1]/h1').text
print("Product Name:", product_name)

Product Name: Project Source  Pro-Flush White Elongated Chair Height 2-piece WaterSense Toilet 12-in Rough-In Size (Ada Compliant)


In [7]:
# Scraping each item's SKU (or unique ID)

sku = driver.find_element(By.XPATH, './/*[@id="pdp-lpd"]/div/div[2]/p[2]').text
print("SKU:", sku)

SKU: Model #MT-802PRO


In [8]:
# Click the show more button to load reviews

time.sleep(sec())
load_review = driver.find_element(By.XPATH, '//*[@id="preview-reviews"]/div[2]/div/button')
load_review.click()

In [9]:
# Create a empty dataframe to collect informations

df = pd.DataFrame(columns = ['subject', 'author', 'date', 'like', 'dislike', 'review'])

In [15]:
# Create a for loop to collect all the reviews

review_list = []
for i in range(5, 15):
    #print(i)
    try: 
        time.sleep(sec()) 
        readmore = driver.find_element(By.XPATH, '//*[@id="rrApp"]/div/div[2]/div/div/div[2]/div['+ str(i) + ']/div[1]/div/div[1]/div/button')
        readmore.click()
    except: 
        pass 
    subject = driver.find_element(By.XPATH, '//*[@id="rrApp"]/div/div[2]/div/div/div[2]/div['+ str(i) + ']/div[1]/div/div[1]/div/p[1]').text
    author = driver.find_element(By.XPATH, '//*[@id="rrApp"]/div/div[2]/div/div/div[2]/div['+ str(i) + ']/div[1]/div/div[2]/div/span[1]').text
    date = driver.find_element(By.XPATH, '//*[@id="rrApp"]/div/div[2]/div/div/div[2]/div['+ str(i) + ']/div[1]/div/div[2]/div/span[2]').text
    
    #like = driver.find_element(By.XPATH, '//*[@id="positive214101389"]/span/span').text
    #dislike = driver.find_element(By.XPATH, '//*[@id="negative214101389"]/span/span').text
    
    review = driver.find_element(By.XPATH, '//*[@id="rrApp"]/div/div[2]/div/div/div[2]/div['+ str(i) + ']/div[1]/div/div[1]/div/p[2]').text
    
    items = {
        'subject': subject,
        'author': author,
        'date': date,
        'review': review
    }
    review_list.append(items)
    
    i += 1

    
like_list = []
fbs = driver.find_elements(By.CLASS_NAME, 'review-row')
for fb in fbs:
    string = fb.find_element(By.CLASS_NAME, 'cgcfbbtncol').text
    like = string[(string.find('(', 0) + 1) : (string.find(')', 0))] # need to improve
    dislike = string[(string.find('(', 4) + 1) : (string.find(')', 4))] # need to improve
    
    like = {
        'like': like,
        'dislike': dislike
    }
    
    like_list.append(like)
    

In [16]:
# Put the lists into dataframe

review_df = pd.DataFrame(review_list)
like_df = pd.DataFrame(like_list)
review_df = review_df.merge(like_df, left_index=True, right_index=True, how='left')
review_df

,subject,author,date,review,like,dislike
0,"For good flush, needs slight modification",James,"January 16, 2022",For the price and look I would give this a 5 s...,32,9
1,,Roxy,"April 15, 2022",Besides the lid being too small. It's a pretty...,8,9
2,Toilet lid cheaply made,Tammy,"March 20, 2021",Toilet easy install. Good overall except lid d...,25,18
3,really a good buy,Texan,"November 8, 2022",I've had this for about a month. I have mostly...,5,0
4,grest toilet for a great price!,TexanStrong,"February 27, 2021","Like most Texans, Valetines day was just a dis...",54,9
5,Poor quality control.,Paul,"June 13, 2021",I bought the chair height versin of this produ...,25,22
6,best buy toilet period!!!,Craig,"June 11, 2021",This toilet is the best buy out there!it's rep...,39,3
7,"A great value, strong flush, and assembled tank!",Jasper,"August 21, 2021",Overall I thought this was a really good value...,24,4
8,Great toilet,camshaft,"June 22, 2022","Toilet works great, easy installation, everyth...",5,0
9,Awesome setup,meridian,"October 22, 2021",I had this toilet for over a month waiting for...,32,1


In [17]:
# Scraping Likes on a product review (how helpful a review was?)

like = driver.find_element(By.XPATH, '//*[@id="positive214101389"]/span/span').text
print("Likes:", like)

Likes: (32)


In [18]:
# Scraping Dislikes on a product review (If a review was not helpful, or not true completely)

dislike = driver.find_element(By.XPATH, '//*[@id="negative214101389"]/span/span').text
print("Dislikes:", dislike)

Dislikes: (9)


In [19]:
# Click the button to load 10 more reviews 
# the problem is it keep loading with no results

time.sleep(sec()) 
readmore = driver.find_element(By.XPATH, '//*[@id="rrApp"]/div/div[2]/div/div/div[2]/div[15]/button')
readmore.click()

In [20]:
# close the whole window of browser 

driver.quit()